# NIMS-KISTI-NVIDIA Hackathon 

# Problem Statement 



## Agenda 
 - Introduction to FourCastNet
 - Configure FourCastNet
 - Typoon Dataset (JMA best track) 
 - ECMWF ERA5 dataset(CDS API)
 - custom interval 
 - inference 
 - post processing 
 




## download dataset with CDS API 

FourCastNet modeled variables
<table align="left" border="1">
  <tr>
    <th>Vertical Level</th>
    <th>Variable</th>
  </tr>
  <tr>
    <td>Surface</td>
    <td>U10, V10, T2M, SP, MSLP</td>
  </tr>
  <tr>
    <td>1000 hPa</td>
    <td>U, V, Z</td>
  </tr>
  <tr>
    <td>850 hPa</td>
    <td>T, U, V, Z, RH</td>
  </tr>
  <tr>
    <td>500 hPa</td>
    <td>T, U, V, Z, RH</td>
  </tr>
  <tr>
    <td>50 hPa</td>
    <td>Z</td>
  </tr>
  <tr>
    <td>Integrated</td>
    <td>TCWV</td>
  </tr>
</table>



# [register copernicus site ](https://cds.climate.copernicus.eu/user/login)

copernicus support CDSAPI to download netcdf /grib2 files 

In [ ]:
# Hyungon Ryu's key. 
#%%file ~/.cdsapirc
#url: https://cds.climate.copernicus.eu/api/v2
#key: 

## manual download script CDS API
- input : typoon list ( start, end date info)
```
typhoon_list = [
{'id': '2221', 'dur': 15, 'start': '2022101700', 'end': '2022102012', 'latitude': 27.0, 'longitude': 156.4, 'lp': 1004, 'hws': 35, 'grade': 3},
{'id': '2222', 'dur': 33, 'start': '2022102600', 'end': '2022110300', 'latitude': 10.1, 'longitude': 134.9, 'lp': 975, 'hws': 60, 'grade': 4},
    # Add more typhoon data here...
]
```
- tasks : download ECWMF ERA5 dataset for FourCastNet ( sl, pl, variables) 
| Vertical Level | Variable                |
|:-------------- |:-----------------------|
| Surface        | U10, V10, T2M, SP, MSLP |
| 1000 hPa       | U, V, Z                |
| 850 hPa        | T, U, V, Z, RH         |
| 500 hPa        | T, U, V, Z, RH         |
| 50 hPa         | Z                      |
| Integrated     | TCWV                   |
 


- Controlled during the span of two months, spanning the end of one month and the beginning of the next, when a typhoon occurs.
example cases : 
```
typhoon_list = [
{'id': '8002', 'dur': 58, 'start': '1980053018', 'end': '1980060100', 'latitude': 8.3, 'longitude': 142.8, 'lp': 960, 'hws': 85, 'grade': 5},

]
```

In [ ]:
import cdsapi
import os
import xarray as xr
import datetime


PL_VARIABLES = ['geopotential', 'relative_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind']
SL_VARIABLES = ['10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature', 'mean_sea_level_pressure', 'surface_pressure', 'total_column_water_vapour']


def generate_pl_script(start_date, end_date, filename_prefix, save_dir):
    save_path = os.path.join(save_dir, f"{filename_prefix}_pl.nc")
    variables = ', '.join([f"'{var}'" for var in PL_VARIABLES])
    script = f"""
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {{
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [{variables}],
        'pressure_level': ['50', '500', '850', '1000'],
        'year': ['{start_date[:4]}'],
        'month': ['{start_date[4:6]}'],
        'day': {list(range(int(start_date[6:8]), int(end_date[6:8]) + 1))},
        'time': ['00:00', '06:00', '12:00', '18:00'],
    }},
    '{save_path}')
"""
    return script


def generate_sl_script(start_date, end_date, filename_prefix, save_dir):
    save_path = os.path.join(save_dir, f"{filename_prefix}_sl.nc")
    variables = ', '.join([f"'{var}'" for var in SL_VARIABLES])
    script = f"""
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {{
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [{variables}],
        'year': ['{start_date[:4]}'],
        'month': ['{start_date[4:6]}'],
        'day': {list(range(int(start_date[6:8]), int(end_date[6:8]) + 1))},
        'time': ['00:00', '06:00', '12:00', '18:00'],
    }},
    '{save_path}')
"""
    return script


def prepare_ERA5_typhoon_dataset(typhoon_list, debug=False, download=False, save_dir='./', custom_interval=None):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    for typhoon in typhoon_list:
        typhoon_id = typhoon['id']
        start_date = typhoon['start']
        end_date = typhoon['end']
        filename_prefix = f"{typhoon_id}_{start_date}_{end_date}"

        if debug:
            print(f"DEBUG : IDX : {typhoon_id} Date : start:{start_date} end:{end_date}")

        if start_date[:6] == end_date[:6]:  # Single month case
            pl_script = generate_pl_script(start_date, end_date, filename_prefix, save_dir)
            sl_script = generate_sl_script(start_date, end_date, filename_prefix, save_dir)

            if debug:
                print("PL Script:")
                print(pl_script)
                print("SL Script:")
                print(sl_script)

            if download:
                exec(pl_script)
                exec(sl_script)
        else:  # Dual month case
            pl_script_first_month = generate_pl_script(start_date, start_date[:6] + '31', filename_prefix + '_first_month', save_dir)
            sl_script_first_month = generate_sl_script(start_date, start_date[:6] + '31', filename_prefix + '_first_month', save_dir)

            # Convert start date to a datetime object
            start_date_dt = datetime.datetime.strptime(start_date, "%Y%m%d%H")

            # Add one month to start date
            start_date_next_month = start_date_dt.replace(month=start_date_dt.month % 12 + 1, day=1)

            # Format as string in the same format as the original dates
            start_date_next_month_str = start_date_next_month.strftime("%Y%m%d%H")

            pl_script_second_month = generate_pl_script(start_date_next_month_str, end_date, filename_prefix + '_second_month', save_dir)
            sl_script_second_month = generate_sl_script(start_date_next_month_str, end_date, filename_prefix + '_second_month', save_dir)

            if debug:
                print("PL Script (First Month):")
                print(pl_script_first_month)
                print("SL Script (First Month):")
                print(sl_script_first_month)
                print("PL Script (Second Month):")
                print(pl_script_second_month)
                print("SL Script (Second Month):")
                print(sl_script_second_month)

            if download:
                exec(pl_script_first_month)
                exec(sl_script_first_month)
                exec(pl_script_second_month)
                exec(sl_script_second_month)

                merge_netcdf_files(filename_prefix, [f"{filename_prefix}_first_month_pl.nc", f"{filename_prefix}_second_month_pl.nc"], f"{filename_prefix}_pl.nc", save_dir)
                merge_netcdf_files(filename_prefix, [f"{filename_prefix}_first_month_sl.nc", f"{filename_prefix}_second_month_sl.nc"], f"{filename_prefix}_sl.nc", save_dir)


def merge_netcdf_files(output_filename, input_filenames, merged_filename, save_dir='./'):
    merged_path = os.path.join(save_dir, merged_filename)
    input_paths = [os.path.join(save_dir, filename) for filename in input_filenames]
    datasets = [xr.open_dataset(path) for path in input_paths]
    merged_dataset = xr.concat(datasets, dim='time')
    merged_dataset.to_netcdf(merged_path)

    # Clean up temporary files
    for path in input_paths:
        os.remove(path)


### download dataset  (debug mode)

In [ ]:
# Example usage
typhoon_list = [
    {'id': '8002', 'dur': 58, 'start': '1980053018', 'end': '1980060100', 'latitude': 8.3, 'longitude': 142.8, 'lp': 960, 'hws': 85, 'grade': 5},
    # Add more typhoon data here...
]

# Run in debug mode
prepare_ERA5_typhoon_dataset(typhoon_list, debug=True, download=False, save_dir='custom_interval')


### download dataset  (actual download mode)

In [ ]:
# Example usage
typhoon_list = [
    {'id': '2211', 'dur': 69, 'start': '2022082718', 'end': '2022090900', 'latitude': 23.8, 'longitude': 151.1, 'lp': 920, 'hws': 105, 'grade': 5}, # HINNAMNO
    #{'id': '2214', 'dur': 47, 'start': '2022091212', 'end': '2022092000', 'latitude': 22.3, 'longitude': 138.7, 'lp': 910, 'hws': 105, 'grade': 5},  # NANMADOL
    # {'id': '1825', 'dur': 49, 'start': '2018092800', 'end': '2018100712', 'latitude': 7.4, 'longitude': 150.9, 'lp': 900, 'hws': 115, 'grade': 5}, # kong-rey
    # {'id': '1826', 'dur': 54, 'start': '2018102018', 'end': '2018110300', 'latitude': 8.4, 'longitude': 160.7, 'lp': 900, 'hws': 115, 'grade': 5}, # yutu
    # Add more typhoon data here...
]

# Run in debug mode
prepare_ERA5_typhoon_dataset(typhoon_list, debug=True, download=True, save_dir='custom_interval')


In [ ]:
!ls -lah custom_interval